In [2]:
import itertools

words = open('../data/names.txt', 'r').read().splitlines()
words[:10]

# words = ["giorgio", "gina"]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
import torch

In [19]:
chars = sorted(list(set(''.join(words))))

In [83]:
inputs = list(itertools.product(chars, repeat=2))
inputs += [('.', c) for c in chars]
inputs[-30:]

[('z', 'w'),
 ('z', 'x'),
 ('z', 'y'),
 ('z', 'z'),
 ('.', 'a'),
 ('.', 'b'),
 ('.', 'c'),
 ('.', 'd'),
 ('.', 'e'),
 ('.', 'f'),
 ('.', 'g'),
 ('.', 'h'),
 ('.', 'i'),
 ('.', 'j'),
 ('.', 'k'),
 ('.', 'l'),
 ('.', 'm'),
 ('.', 'n'),
 ('.', 'o'),
 ('.', 'p'),
 ('.', 'q'),
 ('.', 'r'),
 ('.', 's'),
 ('.', 't'),
 ('.', 'u'),
 ('.', 'v'),
 ('.', 'w'),
 ('.', 'x'),
 ('.', 'y'),
 ('.', 'z')]

In [21]:
stoi = {s:i for i, s in enumerate(inputs)}
itos = {i:s for s, i in stoi.items()}
stoo = {s:i+1 for i, s in enumerate(chars)}
stoo['.'] = 0
otos = {i:s for s, i in stoo.items()}
n_tokens = len(stoi)
n_tokens

702

In [126]:
N = torch.ones((n_tokens, 27), dtype=torch.int32)

In [127]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for i in range(len(chs)-2):
        # print(tuple(chs[i:i+2]), chs[i+2])
        ixi = stoi[tuple(chs[i:i+2])]
        ixo = stoo[chs[i+2]]
        N[ixi, ixo] += 1

In [128]:
P = N.float()
P /= P.sum(1, keepdims=True)

In [11]:
P[676]

tensor([0.3341, 0.0215, 0.0047, 0.0028, 0.0075, 0.0033, 0.0014, 0.0005, 0.2385,
        0.0163, 0.0005, 0.0033, 0.0229, 0.0187, 0.2235, 0.0005, 0.0009, 0.0028,
        0.0359, 0.0322, 0.0103, 0.0005, 0.0028, 0.0009, 0.0014, 0.0028, 0.0098])

In [168]:
g = torch.Generator().manual_seed(2147483647 + 14)
for i in range(10):
    ix = stoi[('.', 'a')] + torch.multinomial(
        N[stoi[('.', 'a')]:stoi[('.', 'z')]+1].float().sum(1),
        num_samples=1,
        replacement=True,
        generator=g
    ).item()
    out = [itos[ix][1]]
    while True:
        p = P[ix]
        ixo = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(otos[ixo])
        # print(itos[ix], ix)
        # print(otos[ixo], ixo)
        if otos[ixo] == '.':
            break
        ix = stoi[itos[ix][1], otos[ixo]]

    print(''.join(out))

ala.
ka.
makacarielylan.
samaroswren.
auoira.
bayse.
elyn.
dijalee.
belain.
arinicto.


In [12]:
# GOAL: to maximise the likelihood of the data w.r.t. the model parameters
# (statistical modelling). what mean this ? we can't realistically get 100%
# probability. let's say:
# our training set is ['giorgio', 'gina']:
# there is no way to have 100% probability for what follows 'i',
# since it could either be bigram 'io' or 'in'. But we want to maximise the
# chance of them showing up, and correctly: as close to 1/3 for 'in' and 2/3
# for 'io' as possible. If we make 'io' much higher, the likelihood of the
# rest of the data ('g<in>a') showing up becomes much lower, and so the
# overall quality of the model is lower. MAXIMISE THE LIKELIHOOD OF THE
# DATA WITH RESPECT TO THE MODEL PARAMETERS. CAN'T HAVE ALL 1s AS
# PROBABILITIES SINCE THEY WILL CLASH - JUST LOOKING FOR WHAT'S BEST. WITH
# BIGRAMS DATA, THE CALCULATED LOSS W.R.T TRAINING DATA IS AS GOOD AS IT'S
# GOING TO GET.
# so we want to maximise the product of all probabilities.
# i.e. maximise the log likelihood (since probs are 0-1, the logs are gonna
# be negative, and we want to get close to 0)
# i.e. minimise negative log likelihood (positive number == loss, want it to
# be as low as possible)
# i.e. minimise average negative log likelihood.

# The model is good when the probabilities for each bigram are high. Why?
# See above.


In [132]:
log_likelihood = 0.0
n = 0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for i in range(len(chs)-2):
        ix1, ix2 = stoi[tuple(chs[i:i+2])], stoo[chs[i+2]]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        # print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood

print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-410414.9688)
nll=tensor(410414.9688)
2.092747449874878


In [41]:
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.nn.functional import cross_entropy, nll_loss, log_softmax

In [158]:
# create the training set of all the trigrams
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for i in range(len(chs)-2):
        ix1 = stoi[tuple(chs[i:i+2])]
        ix2 = stoo[chs[i+2]]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples:', num)

number of examples: 228146


In [36]:
L = 100
R = 0.01

In [37]:
def forward_pass(inputs, weights):
    # xenc = F.one_hot(inputs, num_classes=n_tokens).float() # input to the NN: one-hot enc
    # logits = xenc @ weights # predict log-counts
    logits = weights[inputs]
    counts = logits.exp() # convert to counts (analogous to N above)
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next char

    return probs

In [39]:
def mean_nll(probs, n_inputs, labels):
    return -probs[torch.arange(n_inputs), labels].log().mean() + R*(W**2).mean()

In [47]:
train_ixs, dev_ixs, test_ixs = random_split(range(num), [0.8, 0.1, 0.1])
train_xs, train_ys = xs[train_ixs], ys[train_ixs]
dev_xs, dev_ys = xs[dev_ixs], ys[dev_ixs]
test_xs, test_ys = xs[test_ixs], ys[test_ixs]

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((n_tokens, 27), generator=g, requires_grad=True)

In [28]:
W.shape

torch.Size([702, 27])

In [50]:
# gradient descent
for k in range(100):
    probs = forward_pass(train_xs, W)

    # print(f'{W.shape=}')
    # print(f'{xenc.shape=}')
    # print(f'{probs.shape=}')
    # print(f'{ys.shape=}')
    loss = mean_nll(probs, len(train_xs), train_ys)
    # second arg normalises to bring Ws to 0. equivalent to (N+1) from above.
    # unsure why we do this here though.
    print(f'{loss:=.4f}')

    # second-last 2 lines are softmax = go from logits (some positive & negative
    # numbers) to a probability distribution.

    # backward pass
    W.grad = None  # set gradient to 0
    loss.backward()

    # update
    W.data += -L * W.grad

2.1829
2.1825
2.1820
2.1815
2.1811
2.1806
2.1802
2.1797
2.1793
2.1789
2.1784
2.1780
2.1776
2.1771
2.1767
2.1763
2.1759
2.1755
2.1751
2.1747
2.1743
2.1739
2.1735
2.1731
2.1727
2.1723
2.1720
2.1716
2.1712
2.1708
2.1705
2.1701
2.1697
2.1694
2.1690
2.1687
2.1683
2.1680
2.1676
2.1673
2.1669
2.1666
2.1663
2.1659
2.1656
2.1653
2.1650
2.1646
2.1643
2.1640
2.1637
2.1634
2.1631
2.1627
2.1624
2.1621
2.1618
2.1615
2.1612
2.1609
2.1606
2.1604
2.1601
2.1598
2.1595
2.1592
2.1589
2.1586
2.1584
2.1581
2.1578
2.1575
2.1573
2.1570
2.1567
2.1565
2.1562
2.1560
2.1557
2.1554
2.1552
2.1549
2.1547
2.1544
2.1542
2.1539
2.1537
2.1534
2.1532
2.1529
2.1527
2.1525
2.1522
2.1520
2.1518
2.1515
2.1513
2.1511
2.1508
2.1506


In [51]:
dev_loss = mean_nll(forward_pass(dev_xs, W), len(dev_xs), dev_ys)
print(f'{dev_loss.item()=:.4f}')
test_loss = mean_nll(forward_pass(test_xs, W), len(test_xs), test_ys)
print(f'{test_loss.item()=:.4f}')

dev_loss.item()=2.1896
test_loss.item()=2.1832


In [ ]:
# Softmax & Sigmoid are output layers. Softmax returns a probability distribution - increasing the
# likelihood of one class reduces the likelihood of others (= multiclass classification. Only 1 class).
# Sigmoid gives 0-1 per each class. They could sum to greater than 1. (= multilabel classification.
# multiple classes could be valid).

# You can mix and match output layers with cost functions.

# Cross-entropy and NLL (negative log-likelihood, similar to maximum likelihood estimation) are cost
# functions. In multiclass classification, they are equivalent, because cross-entropy over a softmax
# output layer (i.e. a probability distribution over all examples) is the same as NLL. In multi-label
# classification, you can't have a prob. distribution over all examples (like the NLL equation), so
# there is no equivalence. You can do binary cross-entropy here, interpreting each output neuron as
# its own probability p (YES) and 1-p (NO), then sum over all these probabilities.

In [ ]:
# Cross-entropy and NLL both reward network for high probabilities in correct output classes,
# but Cross-entropy also penalises network for high probabilities in the wrong classes. NLL alone
# doesn't do this, but we achieve something similar using the regularisation factor from above
# (which pushes all weights to 0). I think this only happens in binary cross-entropy. See above.

In [ ]:
# NN approach is converging on the final loss of the table-probability approach
# because the table approach is as good as we can get it. Essentially, it is
# perfect. The problem arises when we expand from bigrams. Tri-grams, 4-grams,
# the tables for these become unwieldy (4-gram: need all 3-char combination
# inputs leading to an output). It is un-scalable.
# NNs (and the gradient descent approach) are scalable - we just change the
# way to get the logits (the forward pass), and everything else stays the same.
# Stick a softmax to the end of it, gradient descent, adjust weights. Easy.

In [ ]:
# WOW OK
# The W we end up with is the table N we calculated by counting from above!!
# The comparison: Matrix multiplying a one-hot encoded vector by a matrix
# is equivalent to 'selecting' the row represented by that vector (if we
# had 5 as the one-hot, we select row 5 of W). Which is what we do with N:
# based on the first character, pick that row in the table, and you have a row
# of counts for each character that could follow it.
# Damnnnnnnnnn.
# Well, W is the log-counts. So W.exp() kinda equals N. Difference is, we
# counted everything to get to N, while we started with random values and
# grad-descented to what W is to get the right logcounts/counts/probabilities.